In [ ]:
#hide
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
import warnings
from nbdev import show_doc
with warnings.catch_warnings():
    warnings.simplefilter("ignore")    

# RFPYE
> Este módulo tem como objetivo o processamento e extração otimizada de dados dos arquivos `.bin` de monitoramento do espectro provenientes do script Logger executados nas estações de Monitoramento CRFS RFeye Node. Para tal utilizamos as várias funcionalidades da biblioteca [fastcore](https://fastcore.fast.ai/basics.html), que expande e otimiza as estruturas de dados da linguagem python. 

## Instalação

`Ubuntu`: 

```bash
python -m pip install rfpye
```

`Windows`:

Como parte dessa lib utiliza código c compilado com `Cython`, é preciso que um compilador `C` esteja instalado. Em Windows, uma opção é instalar a versão apropriada do Visual Studio seguindo as orientações do site da Microsoft. No entanto uma solução mais simples e a recomendada é utilizando o `conda`.

Primeiramente instale o [miniconda](https://docs.conda.io/en/latest/miniconda.html). Com o conda instalado e disponível no seu `PATH` ou através do `Anaconda Prompt` execute o comando:

```bash
conda install -c intel libpython m2w64-toolchain -y

echo [build] > %CONDA_PREFIX%\Lib\distutils\distutils.cfg

echo compiler = mingw32 >> %CONDA_PREFIX%\Lib\distutils\distutils.cfg
```

Depois disso basta instalar normalmente a lib:
`python -m pip install rfpye`

Em Linux normalmente o sistema já possui o compilador `gcc` instalado então basta executar o comando `pip install` acima.

## Como utilizar
Abaixo mostramos as funcionalidades principais dos módulos, utilizando-os dentro de algum outro script ou `REPL`

Precisamos necessariamente de um diretório de entrada, contendo um ou mais arquivos `.bin`
> Mude os caminhos abaixo para suas pastas locais

In [ ]:
from fastcore.xtras import Path
from rfpye.utils import get_files
from rich import print

A função abaixo baixa alguns arquivos de exemplo:

In [ ]:
path = Path(r'binfiles')
if not path.exists() or not len(get_files(path, extensions=['.bin'])):
    path = Path('.')
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --output-document 'rfeye002092_210208_T202310_CRFSBINv.5.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --output-document 'rfeye002092_210208_T203131_CRFSBINv.2.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --output-document 'rfeye002292_210208_T202215_CRFSBINv.4.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --output-document 'rfeye002292_210208_T203238_CRFSBINv.3.bin'


A função `parse_bin` é a função principal que encapsula o processamento dos arquivos bin.

In [ ]:
from rfpye.parser import parse_bin
show_doc(parse_bin)

<h4 id="parse_bin" class="doc_header"><code>parse_bin</code><a href="https://github.com/ronaldokun/rfpye/tree/master/rfpye/parser.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>parse_bin</code>(**`bin_file`**:`Union`\[`str`, `Path`\])

Receives a CRFS binfile and returns a dictionary with the file metadata, a GPS Class and a list with the different Spectrum Classes
A block is a piece of the .bin file with a known start and end and that contains different types of information.
It has several fields: file_type, header, data and footer.
Each field has lengths and information defined in the documentation.
Args:
    bin_file (Union[str, Path]): path to the bin file

Returns:
    Dictionary with the file metadata, file_version, string info, gps and spectrum blocks.

## Extração de Dados

Vamos listar arquivos da última versão do script Logger, **CRFS Bin - Versão 5**

In [ ]:
files = get_files(r'D:\OneDrive - ANATEL\Sensores', extensions=['.bin'])
file = files.shuffle()[0]

In [ ]:
%%time
dados = parse_bin(file)

Wall time: 13.2 s


In [ ]:
print(dados)

{
    'filename': 'rfeye002130_210622_T151702.bin',
    'file_version': 23,
    'string': 'CRFS DATA FILE V023',
    'hostname': 'rfeye002130',
    'method': 'ScriptRFeye2021_v.1',
    'unit_info': 'Stationary',
    'file_number': 0,
    'identifier': 'INFO',
    'gps': GPS Data - Median of Coordinates: -10.68619:-37.43918, Altitude: 210.10 
#Satellites: 11.0 ,
    'spectrum': [SpecData(type=67, thread_id=300, description='PMEC 2021 (Faixa 1 de 10).', 
start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, bw=18457, processing='peak', 
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=310, 
description='PMEC 2021 (Faixa 2 de 10).', start_mega=155.0, stop_mega=165.0, unit='dBm', 
ndata=1024, bw=18457, processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), 
SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de 4).', 
start_mega=50.0, stop_mega=90.0, unit='dBμV/m', ndata=1024, bw=73828, processing='peak', 
antenna_id=0, thresh=-70.5, minimum=-70.5), SpecData(type=67, thread_id=110, description='PRD
2021 (Faixa principal 2 de 4).', start_mega=70.0, stop_mega=110.0, unit='dBμV/m', ndata=1024,
bw=73828, processing='peak', antenna_id=0, thresh=-29.5, minimum=-29.5), SpecData(type=67, 
thread_id=120, description='PRD 2021 (Faixa principal 3 de 4).', start_mega=170.0, 
stop_mega=220.0, unit='dBμV/m', ndata=1280, bw=73828, processing='peak', antenna_id=0, 
thresh=-69.5, minimum=-69.5), SpecData(type=67, thread_id=130, description='PRD 2021 (Faixa 
principal 4 de 4).', start_mega=470.0, stop_mega=700.0, unit='dBμV/m', ndata=5888, bw=73828, 
processing='peak', antenna_id=0, thresh=-42.5, minimum=-42.5), SpecData(type=67, 
thread_id=200, description='PMEF 2021 (Faixa 1 de 6).', start_mega=700.0, stop_mega=960.0, 
unit='dBm', ndata=6656, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=210, description='PMEF 2021 (Faixa 2 de 6).', 
start_mega=1710.0, stop_mega=1980.0, unit='dBm', ndata=6912, bw=73828, processing='peak', 
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=220, 
description='PMEF 2021 (Faixa 3 de 6).', start_mega=2100.0, stop_mega=1002168.999, 
unit='dBm', ndata=1792, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=230, description='PMEF 2021 (Faixa 4 de 6).', 
start_mega=2290.0, stop_mega=2390.0, unit='dBm', ndata=2560, bw=73828, processing='peak', 
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=240, 
description='PMEF 2021 (Faixa 5 de 6).', start_mega=2500.0, stop_mega=2690.0, unit='dBm', 
ndata=4864, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), 
SpecData(type=67, thread_id=250, description='PMEF 2021 (Faixa 6 de 6).', start_mega=3290.0, 
stop_mega=3700.0, unit='dBm', ndata=10496, bw=73828, processing='peak', antenna_id=0, 
thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=320, description='PMEC 2021 
(Faixa 3 de 10).', start_mega=320.0, stop_mega=340.0, unit='dBm', ndata=512, bw=73828, 
processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, 
thread_id=330, description='PMEC 2021 (Faixa 4 de 10).', start_mega=400.0, stop_mega=410.0, 
unit='dBm', ndata=256, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=340, description='PMEC 2021 (Faixa 5 de 10).', 
start_mega=960.0, stop_mega=1001218.999, unit='dBm', ndata=6656, bw=73828, processing='peak',
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=67, thread_id=350, 
description='PMEC 2021 (Faixa 6 de 10).', start_mega=1001388.999, stop_mega=1001428.999, 
unit='dBm', ndata=1280, bw=73828, processing='peak', antenna_id=0, thresh=-147.5, 
minimum=-147.5), SpecData(type=67, thread_id=360, description='PMEC 2021 (Faixa 7 de 10).', 
start_mega=1530.0, stop_mega=1001648.999, unit='dBm', ndata=3072, bw=73828, 
processing='peak', antenna_

A saída da função é um dicionário, com os metadados do arquivo.

## GPS
No entanto as duas chaves mais importantes do dicionário retornado são `gps` e `spectrum`

Se você imprimir a classe retornada pela chave `gps` é retornado um resumo dos seus atributos:

In [ ]:
print(dados['gps'])

GPS Data - Median of Coordinates: -10.68619:-37.43918, Altitude: 210.10 #Satellites: 11.0

> Para extrair os atributos em si de dado objeto e retorná-los todos num dicionário, o módulo utils tem a função auxiliar `getattrs`

In [ ]:
from rfpye.utils import getattrs
show_doc(getattrs)

<h4 id="getattrs" class="doc_header"><code>getattrs</code><a href="https://github.com/ronaldokun/rfpye/tree/master/rfpye/utils.py#L159" class="source_link" style="float:right">[source]</a></h4>

> <code>getattrs</code>(**`obj`**:`Any`, **`attrs`**:`Iterable`\[`T_co`\]=*`None`*)

Receives an object and return the atributes listed in `attrs`, if attrs is None return its public attributes

In [ ]:
print(getattrs(dados['gps']))

{'altitude': 210.1, 'latitude': -10.686185, 'longitude': -37.43918, 'num_satellites': 11.0}

Os atributos listados são os valores consolidados por meio da __mediana__ dos diversos blocos de GPS do arquivo. 

### Dados Brutos de GPS
> Caso desejar a lista original de valores, os atributos são os mesmos mas precedidos de `_`, o que os torna __atributos privados__ em python, isso somente quer dizer que não são explicitados em algus métodos como `getattrs`, pois normalmente não são acessíveis diretamente, mas nada impede que sejam acessados.

In [ ]:
dados['gps']._latitude

(#18116) [-10.686196,-10.686173,-10.686203,-10.686203,-10.686191,-10.686178,-10.686201,-10.686205,-10.686205,-10.686209...]

In [ ]:
dados['gps']._longitude

(#18116) [-37.439175,-37.439172,-37.439143,-37.43916,-37.439191,-37.439171,-37.439177,-37.439192,-37.439172,-37.43917...]

In [ ]:
dados['gps']._altitude

(#18116) [211.4,221.8,213.3,210.1,223.2,211.1,206.6,212.6,213.1,210.5...]

In [ ]:
dados['gps']._num_satellites 

(#18116) [11,11,12,10,11,11,11,12,12,12...]

## Dados de Nível Espectral
Cada arquivo bin normalmente possui vários fluxos de espectro distintos, cada fluxo espectral é uma classe Python, na chave `spectrum` é retornado uma lista com todos os fluxos de espectro.

In [ ]:
fluxos = dados['spectrum']
print(len(fluxos))

20

Vamos investigar um deles:

In [ ]:
fluxo = fluxos[0]

Ao imprimir um fluxo é mostrado informações mínimas sobre o seu conteúdo:

In [ ]:
print(fluxo)

Blocks of Type: 67, Thread_id: 300, Start: 105.0 MHz, Stop: 140.0 MHz

A função `repr` retorna uma representação com todos os metadados do fluxo:

In [ ]:
print(repr(fluxo))

SpecData(type=67, thread_id=300, description='PMEC 2021 (Faixa 1 de 10).', start_mega=105.0, 
stop_mega=140.0, unit='dBm', ndata=3584, bw=18457, processing='peak', antenna_id=0, 
thresh=-147.5, minimum=-147.5)

Qualquer um dos atributos listados podem ser acessados diretamente:

In [ ]:
print(fluxo.description) , print(fluxo.bw)

PMEC 2021 (Faixa 1 de 10).

18457

(None, None)

No entanto o principal atributo de um fluxo de espectro são os valores de nível medidos, os valores medidos são retornados por meio do atributo `levels`:

In [ ]:
print(fluxo.levels)

[[-93.5 -99.5 -90.5 ... -85.  -86.5 -90.5]
 [-93.5 -90.5 -90.5 ... -88.  -88.5 -90. ]
 [-90.5 -86.5 -86.5 ... -90.5 -86.  -85.5]
 ...
 [-90.  -91.  -92.  ... -92.  -93.5 -96.5]
 [-92.  -93.  -95.  ... -92.5 -91.5 -91. ]
 [-87.5 -87.5 -89.5 ... -90.  -94.5 -94. ]]

In [ ]:
print(f'Formato da matriz com os níveis: {fluxo.levels.shape}')

Formato da matriz com os níveis: (9058, 3584)

O nº de linhas da matriz nos dá o número de pontos medidos naquele dado fluxo e as colunas o número de traços no qual o Span ( Stop - Start ) foi dividido. O número de traços pode ser retornada também por meio da função `len`

In [ ]:
print(len(fluxo))

9058

O atributo anterior retorna uma `numpy.ndarray`, que é um formato eficiente para processamento. 

### Medidas de nível como pandas dataframe
No entanto temos adicionalmente o método `.matrix()` que retorna a matriz de dados como um _Pandas Dataframe_ formatada com o tempo da medição de cada traço como índice das linhas e as frequências de cada traço como coluna.

Vamos mostrar as cinco primeiras e cinco últimas linhas e colunas. 

In [ ]:
fluxo.matrix().iloc[:5, :5]

Frequencies,105.000000,105.009768,105.019537,105.029305,105.039073
Time,,,,,
2021-06-22 15:18:01.549012,-93.5,-99.5,-90.5,-90.0,-92.0
2021-06-22 15:19:01.408958,-93.5,-90.5,-90.5,-92.5,-96.5
2021-06-22 15:20:01.149028,-90.5,-86.5,-86.5,-95.0,-86.5
2021-06-22 15:21:00.968982,-90.5,-90.5,-91.5,-89.0,-89.0
2021-06-22 15:22:01.729287,-91.0,-90.0,-86.5,-91.0,-93.0


In [ ]:
fluxo.matrix().iloc[-5:, -5:]

Frequencies,139.960927,139.970695,139.980463,139.990232,140.000000
Time,,,,,
2021-06-28 22:11:01.172916,-103.0,-96.5,-94.0,-97.5,-93.5
2021-06-28 22:12:01.317300,-92.0,-95.0,-93.5,-93.0,-92.0
2021-06-28 22:13:01.742929,-96.0,-92.5,-92.0,-93.5,-96.5
2021-06-28 22:14:01.693040,-94.5,-93.0,-92.5,-91.5,-91.0
2021-06-28 22:15:01.614231,-90.5,-91.0,-90.0,-94.5,-94.0


Novamente, caso desejado acessar todos os atributos de um fluxo no formato de dicionário, basta utilizar a função `getattrs`

In [ ]:
getattrs(fluxo)

{'antenna_id': 0,
 'append': <bound method CrfsSpectrum.append of SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de 4).', start_mega=50.0, stop_mega=90.0, unit='dBμV/m', ndata=1024, bw=73828, processing='peak', antenna_id=0, thresh=-35.5, minimum=-35.5)>,
 'bw': 73828,
 'description': 'PRD 2021 (Faixa principal 1 de 4).',
 'matrix': <bound method CrfsSpectrum.matrix of SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de 4).', start_mega=50.0, stop_mega=90.0, unit='dBμV/m', ndata=1024, bw=73828, processing='peak', antenna_id=0, thresh=-35.5, minimum=-35.5)>,
 'minimum': -35.5,
 'ndata': 1024,
 'processing': 'peak',
 'start_mega': 50.0,
 'stop_mega': 90.0,
 'thread_id': 100,
 'thresh': -35.5,
 'timestamp': (#1812) [numpy.datetime64('2021-06-28T15:35:00.933900'),numpy.datetime64('2021-06-28T15:40:00.730000'),numpy.datetime64('2021-06-28T15:45:00.758300'),numpy.datetime64('2021-06-28T15:50:00.728300'),numpy.datetime64('2021-06-28T15:55:00

### CRFS Bin - Versão 5 - Arquivos Comprimidos
Vamos listar arquivos da última versão do script Logger, Versão 5, arquivos comprimidos onde o piso de ruído é suprimido.

In [ ]:
file = r'binfiles\compressed\rfeye002290_210922_T204046_MaskBroken.bin'

In [ ]:
%%time
compressed = parse_bin(file)

Wall time: 9.97 s


In [ ]:
print(compressed)

{
    'filename': 'rfeye002290_210922_T204046_MaskBroken.bin',
    'file_version': 23,
    'string': 'CRFS DATA FILE V023',
    'hostname': 'rfeye002290',
    'method': 'ScriptRFeye2021_v2.cfg',
    'unit_info': 'Stationary',
    'file_number': 0,
    'identifier': 'INFO',
    'gps': GPS Data - Median of Coordinates: -22.70008:-47.66684, Altitude: 518.40 
#Satellites: 11.0 ,
    'spectrum': [SpecData(type=68, thread_id=321, description='PMEC 2021 (Faixa 3 de 10). @ 
-80dBm, 100kHz.', start_mega=320.0, stop_mega=340.0, unit='dBm', ndata=512, bw=73828, 
processing='peak', antenna_id=0, thresh=-100, minimum=-147.5), SpecData(type=68, 
thread_id=301, description='PMEC 2021 (Faixa 1 de 10). @ -80dBm, 10kHz.', start_mega=108.0, 
stop_mega=137.0, unit='dBm', ndata=14848, bw=3690, processing='peak', antenna_id=0, 
thresh=-100, minimum=-147.5), SpecData(type=68, thread_id=341, description='PMEC 2021 (Faixa 
5 de 10). @ -80dBm, 100kHz.', start_mega=960.0, stop_mega=1001218.999, unit='dBm', 
ndata=6656, bw=73828, processing='peak', antenna_id=0, thresh=-100, minimum=-147.5), 
SpecData(type=68, thread_id=311, description='PMEC 2021 (Faixa 2 de 10). @ -80dBm, 10kHz.', 
start_mega=156.0, stop_mega=163.0, unit='dBm', ndata=3584, bw=3690, processing='peak', 
antenna_id=0, thresh=-100, minimum=-147.5), SpecData(type=68, thread_id=371, 
description='PMEC 2021 (Faixa 8 de 10). @ -80dBm, 100kHz.', start_mega=2690.0, 
stop_mega=1002898.999, unit='dBm', ndata=5376, bw=73828, processing='peak', antenna_id=0, 
thresh=-100, minimum=-147.5), SpecData(type=68, thread_id=351, description='PMEC 2021 (Faixa 
6 de 10). @ -80dBm, 100kHz.', start_mega=1001388.999, stop_mega=1001428.999, unit='dBm', 
ndata=1280, bw=73828, processing='peak', antenna_id=0, thresh=-100, minimum=-147.5), 
SpecData(type=68, thread_id=331, description='PMEC 2021 (Faixa 4 de 10). @ -80dBm, 100kHz.', 
start_mega=400.0, stop_mega=410.0, unit='dBm', ndata=256, bw=73828, processing='peak', 
antenna_id=0, thresh=-100, minimum=-147.5), SpecData(type=68, thread_id=361, 
description='PMEC 2021 (Faixa 7 de 10). @ -80dBm, 100kHz.', start_mega=1530.0, 
stop_mega=1001648.999, unit='dBm', ndata=3072, bw=73828, processing='peak', antenna_id=0, 
thresh=-100, minimum=-147.5)]
}

In [ ]:
fluxo = compressed['spectrum'] ; fluxos

(#8) [SpecData(type=68, thread_id=321, description='PMEC 2021 (Faixa 3 de 10). @ -80dBm, 100kHz.', start_mega=320.0, stop_mega=340.0, unit='dBm', ndata=512, bw=73828, processing='peak', antenna_id=0, thresh=-100, minimum=-147.5),SpecData(type=68, thread_id=301, description='PMEC 2021 (Faixa 1 de 10). @ -80dBm, 10kHz.', start_mega=108.0, stop_mega=137.0, unit='dBm', ndata=14848, bw=3690, processing='peak', antenna_id=0, thresh=-100, minimum=-147.5),SpecData(type=68, thread_id=341, description='PMEC 2021 (Faixa 5 de 10). @ -80dBm, 100kHz.', start_mega=960.0, stop_mega=1001218.999, unit='dBm', ndata=6656, bw=73828, processing='peak', antenna_id=0, thresh=-100, minimum=-147.5),SpecData(type=68, thread_id=311, description='PMEC 2021 (Faixa 2 de 10). @ -80dBm, 10kHz.', start_mega=156.0, stop_mega=163.0, unit='dBm', ndata=3584, bw=3690, processing='peak', antenna_id=0, thresh=-100, minimum=-147.5),SpecData(type=68, thread_id=371, description='PMEC 2021 (Faixa 8 de 10). @ -80dBm, 100kHz.', sta

In [ ]:
fluxo = fluxos[0]
fluxo.matrix().iloc[:5, [0, 1, 2, -3, -2, -1]]

Frequencies,320.000000,320.039139,320.078278,339.921722,339.960861,340.000000
Time,,,,,,
2021-09-22 20:41:05.113032,-87.5,-91.5,-92.5,-87.5,-87.0,-84.5
2021-09-22 20:41:07.283024,-82.0,-82.0,-84.5,-89.0,-84.0,-84.0
2021-09-22 20:41:09.630510,-84.0,-85.0,-88.5,-88.0,-86.5,-87.5
2021-09-22 20:41:13.863041,-85.5,-84.5,-88.5,-85.0,-81.5,-83.0
2021-09-22 20:41:21.673136,-83.5,-85.0,-87.0,-84.5,-85.5,-89.5


In [ ]:
print(len(fluxo))

9788

### CRFS Bin - Versão 4

In [ ]:
file = r'binfiles\rfeye002292_210208_T202215_CRFSBINv.4.bin'
blocks = parse_bin(file)
print(blocks)

{
    'filename': 'rfeye002292_210208_T202215_CRFSBINv.4.bin',
    'file_version': 22,
    'string': 'CRFS DATA FILE V022',
    'hostname': 'rfeye002292',
    'method': 'Script_CRFSBINv4',
    'unit_info': 'Stationary',
    'file_number': 0,
    'identifier': 'LOGGER_VERSION',
    'group_id': 0,
    'text': 'ClearWrite. Peak.',
    'gps': GPS Data - Median of Coordinates: -22.70008:-47.66684, Altitude: 518.30 
#Satellites: 11.0 ,
    'spectrum': [SpecData(type=63, thread_id=20, description='Peak', start_mega=76.0, 
stop_mega=108.0, unit='dBm', ndata=8192, bw=3, processing='peak', antenna_id=0, 
thresh=-147.5, minimum=-147.5), SpecData(type=63, thread_id=30, description='Peak', 
start_mega=70.0, stop_mega=110.0, unit='dBm', ndata=1024, bw=39, processing='peak', 
antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=63, thread_id=10, 
description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, bw=9, 
processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=63, 
thread_id=11, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=9, processing='average', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=63, 
thread_id=12, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=9, processing='peak', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=63, 
thread_id=13, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=9, processing='average', antenna_id=0, thresh=-147.5, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=48, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=51, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=47, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=46, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=59, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=44, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=62, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=50, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=49, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=43, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=52, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=54, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, description='Peak', start_mega=105.0, stop_mega=140.0, unit='dBm', ndata=3584, 
bw=42, processing='average', antenna_id=0, thresh=-90, minimum=-147.5), SpecData(type=64, 
thread_id=14, de

In [ ]:
blocks['spectrum'][0].matrix().iloc[:5, [0, 1, 2, -3, -2, -1]]

Frequencies,76.000000,76.003907,76.007813,107.992187,107.996093,108.000000
Time,,,,,,
2021-02-08 20:22:15.500658,-110.0,-100.5,-99.0,-103.0,-99.0,-96.0
2021-02-08 20:22:16.142770,-105.5,-100.0,-97.5,-94.5,-95.0,-98.0
2021-02-08 20:22:16.500750,-104.0,-102.5,-105.5,-95.5,-98.5,-93.0
2021-02-08 20:22:17.132990,-105.0,-107.0,-103.0,-99.5,-99.5,-102.5
2021-02-08 20:22:17.501352,-97.5,-101.5,-97.0,-104.5,-102.0,-99.5


### CRFS Bin - Versão 3

In [ ]:
file = r'binfiles\rfeye002292_210208_T203238_CRFSBINv.3.bin'
blocks = parse_bin(file)
print(blocks)

{
    'filename': 'rfeye002292_210208_T203238_CRFSBINv.3.bin',
    'file_version': 21,
    'string': 'CRFS DATA FILE V021',
    'hostname': 'rfeye002292',
    'method': 'Script_CRFSBINv3',
    'unit_info': 'Stationary',
    'file_number': 0,
    'identifier': 'LOGGER_VERSION',
    'gps': GPS Data - Median of Coordinates: -22.70008:-47.66684, Altitude: 518.30 
#Satellites: 11.0 ,
    'spectrum': []
}